<img align="right" width="200" height="200" src="https://static.tildacdn.com/tild6236-6337-4339-b337-313363643735/new_logo.png">

# Spark Dataframes I
**Андрей Титов**  
tenke.iu8@gmail.com  

## На этом занятии
+ Сравнение RDD API и DataFrame API 
+ Базовые функции
+ Очистка данных
+ Агрегаты
+ Кеширование 
+ Репартиционирование
+ Встроенные функции
+ Пользовательские функции
+ Соединения
+ Оконные функции

## Сравнение RDD API и DataFrame API 

### Типы данных
**RDD**: низкоуревная распределенная коллекция данных любого типа  
**DF**: таблица со схемой, состоящей из колонок разных типов, описанных в `org.apache.spark.sql.types`  

### Обработка данных
**RDD**: сериализуемые функции  
**DF**: кодогенерация SQL > Java код  

### Функции и алгоритмы
**RDD**: нет ограничений  
**DF**: ограничен SQL операторами, функциями `org.apache.spark.sql.functions` и пользовательскими функциями  

### Источники данных
**RDD**: каждый источник имеет свое API  
**DF**: единое API для всех источников 

### Производительность
**RDD**: напрямую зависит от качества кода
**DF**: встроенные механизмы оптимизации SQL запроса


### Потоковая обработка данных
**RDD**: устаревший DStreams  
**DF**: активно развивающийся Structured Streaming


### Выводы:
+ На текущий момент RDD является низкоуровневым API, которое постепенно уходит "под капот" Apache Spark
+ DF API представляет собой библиотеку для обработки данных с использованием SQL примитивов

## Базовые функции

Создать dataframe можно на основе:
+ локальной коллекции
+ файлов
+ базы данных

In [ ]:
import org.apache.spark.sql.DataFrame

val cityList: Vector[String] = Vector("Moscow", "Paris", "Madrid", "London", "New York")

// метод toDF изначально отсутствует у Vector[T], он добавляется через import spark.implicits._
val df: DataFrame = cityList.toDF

У любого DF есть схема:

In [ ]:
df.printSchema

Посмотреть содержимое DF можно с помощью метода `show()`:

In [ ]:
df.show

Также можно вывести содержимое в вертикальной ориентации - это удобно при большое количестве столбцов:

In [ ]:
df.show(numRows = 20, truncate = 100, vertical=true)

Подсчет количества элементов в DF с помощью `count()`:

In [ ]:
df.count

Отфильтровать данные можно с помощью метода `filter`. В отличие от RDD, он принимает SQL выражение:

In [ ]:
// Требует наличия import spark.implicits._

df.filter('value === "Moscow").show

In [ ]:
// Требует наличия import spark.implicits._

df.filter($"value" === "Moscow").show

In [ ]:
// sugar free & type safe
// Три знака равно здесь используются, тк на самом деле это метод,
// применяемый к колонке org.apache.spark.sql.Column

import org.apache.spark.sql.functions.col

df.filter(col("value") === "Moscow").show

In [ ]:
// легко ошибиться и получить ошибку в рантайме

df.filter("value = 'Moscow'").show

In [ ]:
// промежуточный вариант между col и обычной строкой
// expr также может использоваться для вызова SQL builtin функций, 
// отсутствующих в org.apache.spark.sql.functions

import org.apache.spark.sql.functions.expr

df.filter(expr("value = 'Moscow'")).show

Добавить новую колонку можно с помощью метода `withColumn`. Необходимо помнить, что данный метод, как и другие, является трансформацией и не изменяет оригинальный DF, а создает новый.

In [ ]:
import org.apache.spark.sql.functions.upper
df.withColumn("upperCity", upper('value)).show

Аналогичный результат получить, используя метод `select`. Данный метод может быть использован не только для выбора определенных колонок, но и для создания новых.

In [ ]:
val withUpper = df.select('value, upper('value).alias("upperCity"))
withUpper.show

Если передать `col("*")` в `select`, то вы получите DF со всеми колонками. Это полезно, когда вы не знаете список всех колонок (например вы получили его через API), но вам нужно их все выбрать и добавить новую колонку. Это можно сделать следующим образом:

In [ ]:
// методы name, as и alias часто являются взаимозаменяемыми

import org.apache.spark.sql.functions._

withUpper.select(
    col("*"), 
    lower($"value").name("lowerCity"), 
    (length('value) + 1).as("length"),
    lit("foo").alias("bar")).show

При необходимости в `select` можно передать список колонок, используя обычные строки:

In [ ]:
withUpper.select("value", "upperCity").show

Удалить колонку из DF можно с помощью метода `drop`:

In [ ]:
// drop не будет выдавать ошибку, если будет указана несуществующая колонка

withUpper.drop("upperCity", "abraKadabra").show

### Выводы:
+ методы `filter` и `select` принимают в качестве аргументов колонки [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column). Это может быть либо ссылка на существующую колонку, либо функцию из [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$)
+ любые трансформации возвращают новый DF, не меняя существующий
+ тип [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column) играет важную роль в DF API - на его основе создаются ссылки на существующие колонки, а также функции, принимающие [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column) и возвращающие [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column). По этой причине обычное сравнение `==` не будет работать в DF API, тк `filter` принимает [org.apache.spark.sql.Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column), а не `Boolean`
+ Класс DataFrame в последних версиях Spark представляет собой `org.apache.spark.sql.Dataset[org.apache.spark.sql.Row]`, поэтому его описание следует искать в [org.apache.spark.sql.Dataset](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset)

## Очистка данных

Одной из задач обработки данных является их очистка. DF API содержит класс функций "not available", описанный в пакете [org.apache.spark.sql.DataFrameNaFunctions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.DataFrameNaFunctions). В данном пакете есть три функции:
+ `na.drop`
+ `na.fill`
+ `na.replace`

Для демонстрации работы данных функций создадим новый датасет:

In [ ]:
import org.apache.spark.sql.Column
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Dataset

val testData =
"""{ "name":"Moscow", "country":"Rossiya", "continent": "Europe", "population": 12380664}
{ "name":"Madrid", "country":"Spain" }
{ "name":"Paris", "country":"France", "continent": "Europe", "population" : 2196936}
{ "name":"Berlin", "country":"Germany", "continent": "Europe", "population": 3490105}
{ "name":"Barselona", "country":"Spain", "continent": "Europe" }
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 11922948 }
{ "name":"Cairo", "country":"Egypt", "continent": "Africa", "population": 11922948 }
{ "name":"New York, "country":"USA","""

// Создаем DF из одной строки и добавляем данные в виде новой колонки
val raw = spark.range(0,1).select(lit(testData).alias("value"))

// Создаем новую колонку, разибая наши данные по \n
val jsonStrings: Column = split(col("value"), "\n").alias("value")

// Используем функцию explode для того, чтобы развернуть массив мехом наружу и используем темную магию 
// для превращения DataFrame в Dataset[String]
val splited: Dataset[String] = raw.select(explode(jsonStrings)).as[String]

splited.show(numRows = 10, truncate = false)


// Создаем новый датафре... датасет, в котором наши JSON строки будут распарсены
val df: Dataset[Row] = spark.read.json(splited)
df.printSchema
df.show

Для очистки датасета:
+ удалим строку с навалидным JSON, сохраним ее в отдельное место
+ удалим дубликаты
+ заполним `null`ы в колонках
+ исправим `Rossiya` на `Russia`

In [ ]:
val corruptData = df.select(col("_corrupt_record")).na.drop("all").collect

In [ ]:
val fillData: Map[String, Any] = Map("continent" -> "Undefined", "population" -> 0)
val replaceData: Map[Any, Any] = Map("Rossiya" -> "Russia")

val cleanData = 
    df
    .drop(col("_corrupt_record"))
    .na.drop("all")
    .na.fill(fillData)
    .na.replace("country", replaceData)
    .dropDuplicates


cleanData.show

### Выводы:
+ DF API обладает удобным API для очистки данных, позволяющим разработчику сконцентрироваться разработчику на бизнес логике, а не на написании функций для обработки всех возможных исключительных ситуаций
+ метод `spark.read.json` позволяет читать не только файлы, но и `Dataset[String]`, содержащие JSON строки.

## Агрегаты
Посчитаем суммарное население и количество городов с разбивкой по континентам:

In [ ]:
val aggCount = cleanData.groupBy('continent).count
aggCount.show

In [ ]:
val aggSum = cleanData.groupBy('continent).sum("population")
aggSum.show

Для того, чтобы совместить несколько агрегатов в одном DF, мы можем использовать метод `agg()`. Данный метод позволяет использовать любые `Aggregate functions` из пакета [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$)

In [ ]:
val agg = cleanData.groupBy('continent).agg(count("*").alias("count"), sum("population").alias("sumPop"))
agg.show

С помощью агрегатов мы можем выполнять такие действия, как, например, `collect_list` и `collect_set`. Стоит отметить, что колонки в Spark могут иметь не только скалярные типы, но и структуры, словари и массивы:

In [ ]:
val aggList = cleanData.groupBy('continent).agg(collect_list("country").alias("countries"))
aggList.printSchema
aggList.show(numRows = 10, truncate = 100, vertical = true)

Используя методы `struct` и `to_json`, мы можем превратить произвольный набор колонок в JSON строку. Этот методы часто используется перед отправкой данных в Kafka

In [ ]:
val withStruct = aggList.select(struct('continent, 'countries).alias("s"))
withStruct.printSchema

withStruct.show(10, false)

In [ ]:
withStruct.withColumn("s", to_json('s)).show(10, false)

Если необходимо превратить все колонки DF в JSON String, можно воспользоваться функций `toJSON`:

In [ ]:
val jString: Dataset[String] = aggList.toJSON
jString.show(5, false)

Если нам необходимо создать колонки из значений текущих колонок, мы можем воспользоваться функцией `pivot`

In [ ]:
cleanData.groupBy(col("country")).pivot("continent").agg(sum("population")).show

### Выводы:
+ DF API позволяет строить большое количество агрегатов. При этом необходимо помнить, что операции `groupBy`, `cube`, `rollup` возвращают [org.apache.spark.sql.RelationalGroupedDataset](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.RelationalGroupedDataset), к которому затем необходимо применить одну из функций агрегации - `count`, `sum`, `agg` и т. п.
+ При вычислении агрегатов необходимо помнить, что эта операция требует перемешивания данных между воркерами, что, в случае перекошенных данных, может привести к OOM на воркере.

## Кеширование
По умолчанию при применении каждого действия Spark пересчитывает весь граф, что может негативно сказать на производительности приложения. Для демонстрации возьмем датасет [Airport Codes](https://datahub.io/core/airport-codes)  

In [ ]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "true")
val airports = spark.read.options(csvOptions).csv("/tmp/datasets/airport-codes.csv")
airports.printSchema
airports.show(numRows = 1, truncate = 100, vertical = true)

Посчитаем несколько агрегатов. Несмотря на то, что `onlyRu` является общим для всех действий, он пересчитывается при вызове каждого действия.

In [ ]:
val onlyRuAndHigh = airports.filter('iso_country === "RU" and 'elevation_ft > 1000)
onlyRuAndHigh.show(numRows = 1, truncate = 100, vertical = true)

onlyRuAndHigh.count
onlyRuAndHigh.collect
onlyRuAndHigh.groupBy('municipality).count.orderBy('count.desc).na.drop("any").show

Для решения этой проблемы следует использовать методы `cache`, либо `persist`. Данные методы сохраняют состояние графа после первого действия, и следующие обращаются к нему. Разница между методами заключается в том, что `persist` позволяет выбрать, куда сохранить данные, а `cache` использует значение по умолчанию. В текущей версии Spark это [StorageLevel.MEMORY_ONLY](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence). Важно помнить, что данный кеш не предназначен для обмена данными между разными Spark приложения - он является внутренним для приложения. После того, как работа с данными окончена, необходимо выполнить `unpersist` для очистки памяти

In [ ]:
onlyRuAndHigh.cache
onlyRuAndHigh.count
// при вычислении count данные будут помещены в cache
onlyRuAndHigh.show(numRows = 1, truncate = 100, vertical = true)
onlyRuAndHigh.collect
onlyRuAndHigh.groupBy('municipality).count.orderBy('count.desc).na.drop("any").show

onlyRuAndHigh.unpersist

### Выводы:
+ Использование `cache` и `persist` позволяет существенно сократить время обработки данных, однако следует помнить и об увеличении потребляемой памяти на воркерах

## Репартиционирование
RDD и DF являются представляют собой классы, описывающие распределенные коллекции данных. Они (коллекции) разбиты на крупные блоки, которые называются партициями. В графе вычисления, который называется в Spark DAG (Direct Acyclic Graph), есть три основных компонента - `job`, `stage`, `task`.

`job` представляет собой весь граф целиком, от момента создания DF, до применения `action` к нему. Состоит из одной или более `stage`. Когда возникает необходимость сделать `shuffle` данных, Spark создает новый `stage`. Каждый `stage` состоит из большого количества `task`. `task` это базовая операция над данными. Одновременно Spark выполняет N `task`, которые обрабатывают N партиций, где N - это суммарное число доступных потоков на всех воркерах.

Исходя из этого, важно обеспечивать:
+ достаточное количество партиций для распределения нагрузки по всем воркерам
+ равномерное распределение данных между партициями

Создадим датасет с перекосом данных:

In [ ]:
import org.apache.spark.sql.functions._

val skewColumn = when(col("id") < 900, lit(0)).otherwise(lit(1))

val skewDf = spark.range(0,1000).repartition(10, skewColumn)

def printItemPerPartition[T](ds: Dataset[T]): Unit = {
    ds.mapPartitions { x => Iterator(x.length) }
    .withColumnRenamed("value", "itemPerPartition")
    .show(50, false)
}

printItemPerPartition[java.lang.Long](skewDf)

Любые операции с таким датасетом будут работать медленно, т.к.
+ если суммарное количество потоков на всех воркерах больше 10, то в один момент времени работать будут максимум 10, остальные будут простаивать
+ из 10 партицийи только в 2 есть данные и это означает, что только 2 потока будут обрабатывать данные, при этом из-за перекоса данных между ними (900 vs 100) первый станет bottleneck'ом

Обычно перекошенные датасеты возникают после вычисления агрегатов, оконных функций и соединений, но также могут возникать и при чтении источников.

Для устранения проблемы перекоса данных, следует использовать метод `repartition`:

In [ ]:
// здесь мы передаем только новое количество партиций и Spark выполнит RoundRobinPartitioning
val repartitionedDf = skewDf.repartition(20)

printItemPerPartition[java.lang.Long](repartitionedDf)

In [ ]:
// здесь мы добавляем к числу партиций колонку, по которой необходимо сделать репартиционирование,
// поэтому Spark выполнит HashPartitioning
val repartitionedDf = skewDf.repartition(20, col("id"))

printItemPerPartition[java.lang.Long](repartitionedDf)

<img align="right" width="200" height="200" src="https://pngimage.net/wp-content/uploads/2018/06/соленья-png-4.png">

### Соленья
Часто при вычислении агрегатов приходится работать с перекошенными данными:

In [ ]:
airports.printSchema

airports.groupBy('type).count.orderBy('count.desc)

Поскольку при вычислении агрегата происходит неявный `HashPartitioning` по ключу (ключам) агрегата, то при выполнении определенных условий происходит нехватка памяти на воркере, которую нельзя исправить, не изменив подход к построению агрегата.

Один из вариантов устранение - соление ключей:

In [ ]:
val saltModTen = pmod(round((rand() * 100), 0), lit(10)).cast("int")

val salted = airports.withColumn("salt", saltModTen)
salted.show(numRows = 1, truncate = 200, vertical = true)

Это позволяет нам существенно снизить объем данных в каждой партиции (30к vs 3к):

In [ ]:
val firstStep = salted.groupBy('type, 'salt).count()

firstStep.orderBy('count.desc).show(200, false)

Вторым шагом мы делаем еще один агрегат, суммируя предыдущие значения `count`:

In [ ]:
val secondStep = firstStep.groupBy('type).agg(sum("count").alias("count"))

secondStep.orderBy('count.desc).show(200, false)

Несмотря на то, что мы сделали две группировки вместо одной, распределение данных по воркерам было более равномерным, что позволило избежать OOM на воркерах.

### Выводы:
+ Партиционирование - важный аспект распределенных вычислений, от которого напрямую зависит стабильность и скорость вычислений
+ В Spark всегда работает правило 1 TASK = 1 THREAD = 1 PARTITION
+ Репартиционирование и соление данных позволяет решить проблему перекоса данных и вычислений
+ Важно помнить, что репартиционирование использует дисковую и сетевую подсистемы - обмен данными происходит **по сети**, а результат записывается **на диск**, что может стать узким местом при выполнении репартиционирования

## Встроенные функции
Помимо базовых SQL операторов, в Spark существует большой набор встроенных функций:
+ API методы из [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$)
+ [SQL built-in functions](https://spark.apache.org/docs/latest/api/sql/index.html)

In [ ]:
val df = spark.range(0,10)

// используем org.apache.spark.sql.functions
val newCol: Column = pmod(col("id"), lit(2))
df.withColumn("pmod", newCol).show

In [ ]:
import org.apache.spark.sql.functions.expr

// используем SQL built-in functions
val newCol: Column = expr("""pmod(id, 2)""")
df.withColumn("pmod", newCol).show

### Выводы
+ Spark обладает широким набором функций для работы с колонками разных типов, включая простые типы - строки, числа, и т. д., а также словари, массивы и структуры
+ Встроенные функции принимают колонки `org.apache.spark.sql.Column` и возвращают `org.apache.spark.sql.Column` в большинстве случаев
+ Встроенные функции доступны в двух местах - org.apache.spark.sql.functions и SQL built-in functions
+ Встроенные функции можно (и нужно) использовать вместе - на вход во встроенные функции могут подаваться результаты встроенной функции, тк все они возвращают `sql.Column` 

### Пользовательские функции

В том случае, если функционала встроенных функций не хватает, можно написать пользовательскую функцию - UDF. Пользовательская функция может принимать до 16 аргументов. Соответствие Spark и Scala типов описано [здесь](https://spark.apache.org/docs/latest/sql-reference.html#data-types)

Необходимо помнить, что `null` в Spark превращается в `null` внутри UDF

In [ ]:
import org.apache.spark.sql.functions.{udf, col}

val df = spark.range(0,10)

val plusOne = udf { (value: Long) => value + 1 }

df.withColumn("idPlusOne", plusOne(col("id"))).show(10, false)

Пользовательская функция может возвращать:
+ простой тип - `String`, `Long`, `Float`, `Boolean` и т.д.
+ массив - любые коллекции, наследующие `Seq[T]` - `List[T]`, `Vector[T]` и т. д.
+ словарь - `Map[A,B]`
+ инстанс `case class`'а
+ Option[T]

Реализуем функцию, которая возвращает имя хоста, на котором работает воркер:

In [ ]:
import java.net.InetAddress

val hostname = udf { () => InetAddress.getLocalHost().getHostName() }

df.withColumn("hostname", hostname()).show(10, false)

Мы также можем использовать монады `Try[T]` и `Option[T]` и для написания пользовательской функции:

In [ ]:
import scala.util.Try
import org.apache.spark.sql.functions.{udf, col}

val df = spark.range(0,10)

val divideTwoBy = udf { (inputValue: Long) => Try(2L / inputValue).toOption }

val result = df.withColumn("divideTwoBy", divideTwoBy(col("id")))
result.printSchema
result.show(10, false)

### Выводы
+ Пользовательские функции позволяют реализовать произвольный алгоритм и использовать его в DF API
+ Пользовательские функции работают медленнее встроенных, поскольку при использовании встроенных функций Spark использует ряд оптимизаций, например векторизацию вычислений на уровне CPU

## Соединения

Join'ы позволяют соединять два DF в один по заданным условиям.

По типу условия join'ы делятся на:
+ equ-join - соединение по равенству одного или более ключей
+ non-equ join - соединение по условию, отличному от равенства одного или более ключей

По методу соединения join'ы бывают:
![Joins](http://kirillpavlov.com/images/join-types.png)
[Источник](http://kirillpavlov.com/blog/2016/04/23/beyond-traditional-join-with-apache-spark/)

Добавим новую колонку к датасету `airports`, в которой будет процент заданного типа аэропорта ко всем типам аэропорта по каждой стране. Первым шагом посчитаем число аэропортов каждого типа по стране:

In [ ]:
import org.apache.spark.sql.functions.{count, round, lit}

val aggTypeCountry = airports.groupBy('type, 'iso_country).agg(count("*").alias("cnt_country_type"))

aggTypeCountry.show(5, false)

Теперь посчитаем количество аэропортов по каждой стране:

In [ ]:
val aggCountry = airports.groupBy('iso_country).agg(count("*").alias("cnt_country"))
aggCountry.show(5, false)

Соединим получившиеся датасеты и получим процентное распределение типов аэропорта по стране

In [ ]:
val percent = 
    aggTypeCountry
        .join(aggCountry, Seq("iso_country"), "inner")
        .select('iso_country, 'type, (round(lit(100) * 'cnt_country_type / 'cnt_country, 2).alias("percent")))
percent.show(5, false)

Соединим полученный датасет с изначальным:

In [ ]:
val result = airports.join(percent, Seq("iso_country", "type"), "left")
result.select('ident, 'iso_country, 'type, 'percent).sample(0.2).show(20, false)

Во всех наших джойнах присутствует массив `Seq[String]`. Это синтаксических сахар, позволяющий не переименовывать колонки датасетов, а просто указать, что соединение будет делаться по колонкам с именами, входящим в массив.

В общем случае условие джойна должно быть выражено в виде колонки `sql.Column`, например:

In [ ]:
import org.apache.spark.sql.Column
val joinCondition: Column = col("left_a") === col("right_a") and col("left_b") === col("right_b")

При этом в данном выражении допускается использование встроенных функций, пользовательских функций и операторов сравнения. Однако следует помнить, что мы выполняем джойн двух распределенных датасетов и если условие соединения будет плохо составлено, то Spark выполнит `cross join`, производительность которого будет "крайне мала" &copy;

### Выводы:
+ Spark поддерживает большое число типов соединений
+ Условием соединения может быть `Seq[String]`, либо `sql.Column`
+ При использовании сложных условий соединения следует избегать тех, которые приведут к `cross join`

## Оконные функции
Оконные функции позволяют делать функции над "окнами" (кто бы мог подумать) данных

Окно создается из класса [org.apache.spark.sql.expressions.Window](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.expressions.Window) с указанием полей, определяющих границы окон и полей, определяющих порядок сортировки внутри окна:

```val window = Window.partitionBy("a", "b").orderBy("a")```

Применяя окна, можно использовать такие полезные функции из [org.apache.spark.sql.functions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$), как ```lag()``` и ```lead()```, а также эффективно работать с данными time-series данными.

Выполним задачу с вычисление процента отношения типов аэропортов, используя оконные функции.

In [ ]:
import org.apache.spark.sql.expressions.Window

val windowCountry = Window.partitionBy("iso_country")
val windowTypeCountry = Window.partitionBy("type", "iso_country")

val result = airports
                .withColumn("cnt_country", count("*").over(windowCountry))
                .withColumn("cnt_country_type", count("*").over(windowTypeCountry))
                .withColumn("percent", round(lit(100) * 'cnt_country_type / 'cnt_country, 2))
                            
result.select('ident, 'iso_country, 'type, 'percent).sample(0.2).show(20, false)

### Выводы:
+ Оконные функции позволяют применять функции, применительно к окнам данных
+ Окно определяется списком колонок и сортировкой
+ Применение оконных функций приводит к `shuffle`

После завершения работы не забывайте останавливать `SparkSession`, чтобы освободить ресурсы кластера!

In [ ]:
spark.stop